In [137]:
# IMPORTING PACKAGES
# ------------------------------------------------------
import pandas as pd
import numpy as np
import os
import re
import warnings

# VISUALIZATION
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = 0
pd.options.display.max_colwidth =20

# CONFIGURATIONS
# ------------------------------------------------------
warnings.filterwarnings("ignore")

# MODEL CONFIG
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

In [112]:
os.chdir("data")

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'data'

In [113]:
df = pd.read_csv("DatafinitiElectronicsProductsPricingData_Cleaned_dataset.csv")

In [114]:
df.columns


Index(['id', 'Actual_price', 'Discount_price', 'Product_availability',
       'Condition', 'Currency', 'Country', 'isSale', 'merchant', 'brand',
       'Description', 'Category_name', 'primaryCategories', 'name',
       'weight_in_kg', 'Impression', 'Day', 'Month', 'Year', 'Week',
       'Month_Name', 'Day_Name', 'Formatted_date', 'Disc_Per', 'Total_sale'],
      dtype='object')

In [115]:
df["Category_name"].value_counts()

drive, storage, hard               892
tv, television, led                824
laptop, computer                   626
tv, 4k, television                 604
car, gps, dash                     603
camera, shoot, point               524
tablet, computer, ipad             517
camera, mirrorless                 500
receiver, amplifier, home          471
headphone, earbud, bluetooth       468
headphone                          429
laptop, computer, macbook          408
car, speaker, subwoofer            370
player, ipod                       363
speaker, portable, bluetooth       327
speaker, audio                     326
sound, speaker, home               318
case, computer                     316
monitor, computer                  314
networking, modem, router          264
camera, camcorder, action          241
memory, computer, component        240
tv, streaming, medium              235
phone, cell                        225
keyboard, computer, mouse          224
speaker, bluetooth, mobil

## Price Elasticity for all Products.

In [116]:
df["Year"].value_counts()

2017    9631
2018    3893
2016     566
2015     445
2014      22
Name: Year, dtype: int64

In [117]:
#dfx = df[df["Category_name"].str.contains("speaker",case=False)]
#dfx= df[(df["Year"]==2017) | (df["Year"]==2018)]
dfx=df

In [118]:
dfx = dfx.reset_index(drop=True)

In [119]:
dfx.sort_values(by=['Formatted_date'],ignore_index=True)

,id,Actual_price,Discount_price,Product_availability,Condition,Currency,Country,isSale,merchant,brand,Description,Category_name,primaryCategories,name,weight_in_kg,Impression,Day,Month,Year,Week,Month_Name,Day_Name,Formatted_date,Disc_Per,Total_sale
0,AVpfnLGdilAPnD_x...,29.39,29.39,Yes,NaN,USD,USA,False,Amazon.com,Sony,Used:AC Adapters...,"camera, camcorde...",Electronics,Sony - Travel Ch...,0.113398,1,30,5,2014,22,May,Friday,2014-05-30,0.0,29.39
1,AVpfnLGdilAPnD_x...,59.99,59.99,Yes,NaN,USD,USA,False,Amazon.com,Sony,Used:AC Adapters...,"camera, camcorde...",Electronics,Sony - Travel Ch...,0.113398,1,30,5,2014,22,May,Friday,2014-05-30,0.0,59.99
2,AVpfcuf7LJeJML43...,1048.00,1048.00,Yes,NaN,USD,USA,False,Amazon.com,Canon,"Electronics,Came...","camera, mirrorless",Electronics,Canon Black EOS ...,3.143393,1,23,8,2014,34,August,Saturday,2014-08-23,0.0,1048.00
3,AVpgzp3F1cnluZ0-...,179.99,179.99,Yes,New,USD,USA,False,Bestbuy.com,AudioQuest,"Speaker Cables,A...","cable, tv, theater",Electronics,AudioQuest - Syd...,3.265862,1,30,8,2014,35,August,Saturday,2014-08-30,0.0,179.99
4,AVpfqlKoilAPnD_x...,251.80,251.80,Yes,New,USD,USA,False,Bestbuy.com,VINPOWER DIGITAL...,"Electronics,Comp...","drive, dvd, comp...",Electronics,ECON-S3T-DVD-BK ...,3.583377,1,31,8,2014,35,August,Sunday,2014-08-31,0.0,251.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14552,AVpe6jFBilAPnD_x...,704.95,704.95,Special,New,USD,USA,False,bhphotovideo.com,Elite Screens,"Office,Fixed Fra...","screen, projecto...",Electronics,Aeon 71.5 x 130....,14.424226,2,25,7,2018,30,July,Wednesday,2018-07-25,0.0,1409.90
14553,AVphtx6BilAPnD_x...,247.99,247.99,Yes,New,USD,USA,False,bhphotovideo.com,Panasonic,"Cameras & Photo,...","lens, camera",Electronics,Lumix G 25mm f/1...,2.000341,12,25,7,2018,30,July,Wednesday,2018-07-25,0.0,2975.88
14554,AV1YHuTsglJLPUi8...,896.00,896.00,Yes,New,USD,USA,False,bhphotovideo.com,Sony,"Digital Cameras,...","camera, mirrorless",Electronics,Alpha a5100 Mirr...,4.526848,3,25,7,2018,30,July,Wednesday,2018-07-25,0.0,2688.00
14555,AVqVHMYrv8e3D1O-...,159.95,159.95,Yes,New,USD,USA,False,bhphotovideo.com,Tenba,"Photography,Came...","camera, bag",Electronics,DNA 15 Slim Mess...,1.315417,4,25,7,2018,30,July,Wednesday,2018-07-25,0.0,639.80


In [120]:
#Grouping data by name and Week number.
#dfx_mod = dfx.groupby(['name', "Month"]).agg({'Disc_Per':'mean','Impression':'sum',"Category_name":lambda x: pd.Series.mode(x)[0],"Formatted_date":lambda x: pd.Series.mode(x)[0]}).reset_index()
dfx_mod = dfx

In [121]:
#Rearranging
#dfx_mod = dfx_mod[["name","Category_name","Disc_Per","Impression","Month","Formatted_date"]]

In [122]:
dfx_mod['Date']= pd.to_datetime(dfx_mod['Formatted_date'])

In [123]:

dfx_mod.sort_values("Month",ignore_index=True,inplace=True)

In [124]:
# sub_df.info()
# converting object data type to int
dfx_mod['Condition'] = np.where(dfx_mod['Condition'] == 'new',1,
                      np.where(dfx_mod['Condition'] == 'refurbished',2,
                      np.where(dfx_mod['Condition'] == 'used',0,0)))


dfx_mod['Product_availability'] = np.where(dfx_mod['Product_availability'] == 'yes',1,
                                 np.where(dfx_mod['Product_availability'] == 'no',0,
                                 np.where(dfx_mod['Product_availability'] == 'other',2,2)))


dfx_mod['Currency']= np.where(dfx_mod['Currency'] == 'USD','1',
                np.where(dfx_mod['Currency'] == 'CAD',"2",
                np.where(dfx_mod['Currency'] == 'SGD',"3",
                np.where(dfx_mod['Currency'] == 'EUR',"4",
                np.where(dfx_mod['Currency'] == 'GBP',"5","6"
                )))))

In [125]:
# we will drop this column
# df['name'].value_counts()
l1=[]
t2=[l1.append(i) for i in dfx_mod['name'].unique() if i not in l1 ] 
t1=[dfx_mod['name'].replace({i:l1.index(i)},inplace=True) for i in dfx_mod['name'] if i in l1]

In [126]:
dfx_mod.columns

Index(['id', 'Actual_price', 'Discount_price', 'Product_availability',
       'Condition', 'Currency', 'Country', 'isSale', 'merchant', 'brand',
       'Description', 'Category_name', 'primaryCategories', 'name',
       'weight_in_kg', 'Impression', 'Day', 'Month', 'Year', 'Week',
       'Month_Name', 'Day_Name', 'Formatted_date', 'Disc_Per', 'Total_sale',
       'Date'],
      dtype='object')

In [127]:
dfx_mod = dfx_mod[["name","Actual_price","Product_availability","Condition","isSale","Currency","Month","Discount_price"]]


### FINAL DATA FRAME

In [128]:
dfx_mod

,name,Actual_price,Product_availability,Condition,isSale,Currency,Month,Discount_price
0,0,699.99,2,0,False,1,1,699.99
1,1,6999.99,2,0,False,1,1,6999.99
2,2,84.99,2,0,True,1,1,78.00
3,3,179.99,2,0,True,1,1,143.64
4,3,129.99,2,0,False,1,1,129.99
...,...,...,...,...,...,...,...,...
14552,647,89.99,2,0,False,1,12,89.99
14553,533,139.33,2,0,False,1,12,139.33
14554,533,124.99,2,0,False,1,12,124.99
14555,1124,397.99,2,0,False,1,12,397.99


#### X-Values = PRICE, Actua_price, Product_availablity, condition, isSale,product_name, week_no

In [132]:
# X data
x_val = dfx_mod.drop('Discount_price', axis=1)

#### Y-Values = Discount Price

In [134]:
# Y data
y_val = dfx_mod['Discount_price']

### Train,Test Splitter

In [138]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(x_val, y_val, test_size=0.2, random_state=42)


### Linera Model

In [140]:
# Train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [237]:
ls = x_val.columns
coeff = pd.DataFrame(model.coef_, columns={"coeff"})
coeff["name"]= ls
coeff


,coeff,name
0,9.652770e-03,name
1,9.591594e-01,Actual_price
2,-1.456613e-13,Product_availabi...
3,1.136868e-13,Condition
4,-2.494498e+02,isSale
5,-4.026665e+00,Currency
6,-5.428476e-01,Month


### Evaluation

In [143]:
def evaluater(eval_type,mod,xval,yval):
    y_pred = mod.predict(xval)

    print(eval_type.center(70,"*"))
    mae = mean_absolute_error(yval, y_pred)
    mse = mean_squared_error(yval, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(yval, y_pred)

    # Print the evaluation metrics
    print('Mean absolute error (MAE): {:.2f}'.format(mae))
    print('Mean squared error (MSE): {:.2f}'.format(mse))
    print('Root mean squared error (RMSE): {:.2f}'.format(mae))
    print('R-squared (R2) score: {:.2f}'.format(r2)) 
    print("*"*70)

In [144]:
evaluater("Train data evaluation",model,X_train,y_train)

************************Train data evaluation*************************
Mean absolute error (MAE): 39.52
Mean squared error (MSE): 16107.92
Root mean squared error (RMSE): 39.52
R-squared (R2) score: 0.98
**********************************************************************


In [145]:
evaluater("Test data evaluation",model,X_test,y_test)

*************************Test data evaluation*************************
Mean absolute error (MAE): 44.46
Mean squared error (MSE): 25823.42
Root mean squared error (RMSE): 44.46
R-squared (R2) score: 0.97
**********************************************************************


In [ ]:
### Model 

 ## Price elasticity = Change in Quanity sold / Change in the Price

The negative sign in the formula for price elasticity of demand (PED) is used to indicate the inverse relationship between the price of a good or service and the quantity demanded by consumers. In other words, as the price of a good or service increases, the quantity demanded by consumers decreases, and vice versa.

The PED formula is:

PED = (% change in quantity demanded / % change in price)

By convention, the price elasticity of demand is expressed as a negative number, as the percentage change in quantity demanded and price usually have opposite signs. For example, if the price of a good increases by 10%, the quantity demanded may decrease by 5%, resulting in a PED of -0.5. The negative sign indicates that the demand for the good is responsive to changes in price and that the quantity demanded decreases as the price increases.

However, in some cases, the demand for a good may be inelastic, which means that the PED is less than 1. In this case, the percentage change in quantity demanded may be smaller than the percentage change in price, resulting in a positive PED value. But, in general, the negative sign is used to indicate the inverse relationship between price and quantity demanded.